# Express Template

## Initialization

### Log4JS
- [NPM](https://www.npmjs.com/package/log4js)
- [GitHub](https://www.npmjs.com/package/log4js)

In [ ]:
process.env.LOG4JS_DEFAULT_LEVEL = "debug";
import * as log4js from "log4js";
import {
  getPackageRelativeFilename,
  defaultLog4JSConfiguration,
  getPackageRoot,
} from "@iamsupercb/common";
var logger = log4js.getLogger(getPackageRelativeFilename("jupyter"));
log4js.configure(defaultLog4JSConfiguration);
logger.debug("==  begins ==");
logger.debug("debug");
logger.info("info");
logger.warn("warn");
logger.error("error");
logger.fatal("fatal");
logger.debug("==  ends ==");

### Environment variables

In [ ]:
logger.debug('== begins ==');
logger.info(`process.env.USER = ${process.env.USER}`);
logger.info(`process.env.HOME = ${process.env.HOME}`);
logger.info(`process.env.AAD_TENANT_ID = ${process.env.AAD_TENANT_ID}`);
logger.info(`process.env.AAD_CLIENT_ID = ${process.env.AAD_CLIENT_ID}`);
logger.info(`process.env.AAD_ENDPOINT_HOST = ${process.env.AAD_ENDPOINT_HOST}`);
logger.info(
  `process.env.MSAL_HOME_ACCOUNT_ID = ${process.env.MSAL_HOME_ACCOUNT_ID}`
);
logger.debug('== ends ==');

### Packages
- [fs](https://nodejs.org/docs/latest/api/path.html)
- [path](https://nodejs.org/docs/latest/api/path.html)
- [@azure/msal-node](https://github.com/AzureAD/microsoft-authentication-library-for-js)

In [ ]:
logger.debug('== begins ==');
import Path from 'path';
import FS from 'fs';
import * as MSAL from '@azure/msal-node';
import * as fosMSAL from '@iamsupercb/msal';
import * as Axios from 'axios';
import HTTPS from "https";
import { IResultSet, IScenario, ScenarioStatus } from "@iamsupercb/jupyter";
logger.debug('== ends ==');

### Globals

In [ ]:
logger.debug("== begins ==");
var resultSet: IResultSet = {
  overwrite: true,
  notebookName: "template",
  description: "template for FOM jupyter notebook that calls express API using msal authentication",
  startDatetime: new Date().toISOString(),
  dataPath: Path.join(process.env.HOME, "data", "fom", "typescript", "express"),
  me:{},
  scenarios: [],
};
if (!FS.existsSync(resultSet.dataPath)) {
  FS.mkdirSync(resultSet.dataPath, { recursive: true });
}
logger.debug(`resultSet = ${JSON.stringify(resultSet)}`);
logger.debug("== getToken begins ==");
let pca = await fosMSAL.init(fosMSAL.defaultConfiguration);
if (!pca) {
  throw new Error("pca is null");
}
let tokenCache = await pca.getTokenCache();
let accountInfo = await tokenCache.getAccountByHomeId(
  process.env.MSAL_HOME_ACCOUNT_ID
);
logger.info(`name: ${accountInfo.name}`);
logger.info(`username = ${accountInfo.username}`);
logger.info(`tenantId = ${accountInfo.tenantId}`);
logger.info(`homeAccountId = ${accountInfo.homeAccountId}`);
let token = await pca.acquireTokenSilent({
  scopes: ['https://graph.microsoft.com/.default'],
  account: accountInfo,
});
logger.info("resultSet.token acquired...");
logger.debug("== getToken ends ==");
resultSet.axios = Axios.default.create({
  baseURL: "https://localhost:3000/api/v1",
  headers: {
    Authorization: `Bearer ${token.idToken}`,
  },
  httpsAgent: new HTTPS.Agent({
    rejectUnauthorized: false,
  }),
});
logger.debug("== ends ==");


### Debug

In [ ]:
logger.debug('== begins ==');
let response = await resultSet.axios.get("/common/me");
resultSet.me = response.data;
logger.info(`me = ${JSON.stringify(resultSet.me)}`);
logger.debug('== ends ==');